In [ ]:
from result_saver import SaverProvider
provider = SaverProvider()

In [ ]:
# parameter grid
distances = [10, 20, 30, 40, 45, 50, 55]
devices = ["ibm_sherbrooke", "ibm_brisbane", "ibm_nazca", "ibm_cusco"]
logicals = ['0', '1']

In [ ]:
from tqdm import tqdm

from qiskit_qec.circuits import RepetitionCodeCircuit
from qiskit import transpile

from soft_info import get_repcode_layout
from Scratch import metadata_helper


# Initialize a dictionary to store basis gates for each device
device_basis_gates = {}

# Loop to populate the device_basis_gates dictionary
for device in devices:
    backend = provider.get_backend(device)
    device_basis_gates[device] = backend.configuration().basis_gates

print(device_basis_gates)


In [ ]:
from soft_info import find_longest_path_in_hex

hex_backend = provider.get_backend('ibm_sherbrooke')
path, _, _  = find_longest_path_in_hex(hex_backend)

# Initialize dictionaries to store transpiled circuits and metadata
transpiled_circuits = {}
metadata = {}

# First loop to generate and store transpiled circuits and metadata
for i, device in enumerate(devices):
    print(f"{i}th Device: {device} out of {len(devices)} devices")
    for distance in tqdm(distances):
        bounded_path = path[:2 * distance - 1]
        layout = bounded_path[1::2] + bounded_path[::2]
        code = RepetitionCodeCircuit(distance, distance)

        for logical in logicals:
            qc = code.circuit[logical]

            # Store the transpiled circuit and metadata in dictionaries
            key = (device, distance, logical)        
            transpiled_qc = transpile(qc, hex_backend, initial_layout=layout,
                                      optimization_level=3, basis_gates=device_basis_gates[device])
            transpiled_circuits[key] = transpiled_qc


In [ ]:
# Second loop to execute jobs per device
for device in tqdm(devices):
    backend = provider.get_backend(device)
    for (dev_key, distance, logical) in transpiled_circuits.keys():  # Unpacking the tuple key
        if dev_key != device:
            continue  # Skip circuits not intended for this device
        transpiled_qc = transpiled_circuits[(dev_key, distance, logical)]

        n_shots = int(1e6 / distance**2)  # Ensure n_shots is an integer
        backend_metadata = metadata_helper(descr="Run bigger Repetition codes v3: default_gates",
                                           code="RepetitionCodeCircuit",
                                           distance=distance,
                                           rounds=distance,
                                           logical=logical,
                                           layout='default_backend: sherbrooke')

        backend.run(backend_metadata, transpiled_qc, shots=n_shots,
                          meas_level=1, meas_return='single')

# Slow triple loop

In [ ]:


# innefficent because most retrieve provider not needed
for distance in tqdm(distances):    
    for device in devices:
        for logical in logicals:
            code = RepetitionCodeCircuit(distance, distance)
            qc = code.circuit[logical]

            backend = provider.get_backend(device)
            layout = get_repcode_layout(distance, backend, _is_hex=True, plot=False)
            transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=3)

            n_shots = 1e6 / distance**2
            metadata = metadata_helper(descr="Run bigger Repetition codes" ,code="RepetitionCodeCircuit", distance=distance, rounds=distance, logical=logical, layout='_is_hex=True')
            backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single') # meas_level=1 for IQ  
            
